In [6]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit import Parameter, ParameterVector
plt.rcParams.update({"font.size": 16})  # enlarge matplotlib fonts
np.set_printoptions(precision=3, suppress=True)
from scipy.linalg import expm
from IPython.display import display, Math

# Define gates
I = np.array([[1, 0],
              [0, 1]], dtype=complex)
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Y = np.array([[0, -1j],
              [1j, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)
ket_0 = np.array([[1],
                  [0]], dtype=complex)

ket_1 = np.array([[0],
                  [1]], dtype=complex)

In [ ]:
# Define gates
I = np.array([[1, 0],
              [0, 1]], dtype=complex)
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Y = np.array([[0, -1j],
              [1j, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)
ket_0 = np.array([[1],
                  [0]], dtype=complex)

ket_1 = np.array([[0],
                  [1]], dtype=complex)

initial_state_bell = (1/np.sqrt(2))*np.kron(ket_0,ket_0) + (1/np.sqrt(2))*np.kron(ket_1,ket_1)
initial_state_10 = np.kron(ket_1,ket_0)
# Parameters
eps = 0
t = 0
delta = 0
time = 0.5

# Hamiltonian
H_analytical = (eps/2)*(np.kron(I, I) - np.kron(I, Z)) \
  + (eps/2)*(np.kron(I, I) - np.kron(Z, I)) \
  + ((t+delta)/2)*np.kron(X, X) \
  + ((t-delta)/2)*np.kron(Y, Y) \

U_analytical = expm(-1.0j * H_analytical * time)

final_state_10 = U_analytical @ initial_state_10
final_state_bell = U_analytical @ initial_state_bell


n_is1 = (1/2)*(np.kron(I, I) - np.kron(Z, I))
n_is2 = (1/2)*(np.kron(I, I) - np.kron(I, Z))
cc_op = (-0.25*np.kron(X,X)-0.25j*np.kron(X,Y)-0.25j*np.kron(Y,X) + 0.25*np.kron(Y,Y))
cc_dagger_op = 1/4*(0.25*np.kron(X,X)-0.25j*np.kron(X,Y)-0.25*np.kron(Y,X) -0.25*np.kron(Y,Y))
gamma_op = 1j*(np.kron(Z,I))

n = (np.abs(final_state_10.conj().T @ n_is1 @ final_state_10))
n2 = (np.abs(final_state_10.conj().T @ n_is2 @ final_state_10))
c = (np.abs(final_state_10.conj().T @ cc_op @ final_state_10))
c_dag = (np.abs(final_state_10.conj().T @ cc_dagger_op @ final_state_10))
gamma = (np.abs(final_state_10.conj().T @ gamma_op @ final_state_10))
E = (np.abs(final_state_10.conj().T @ H_analytical @ final_state_10))
print('ket-10:')
print(f'n_1 = {n}, n_2 = {n2}, gamma = {gamma}, cc = {c}, cc_dag = {c_dag}, E = {E}')

n = (np.abs(final_state_bell.conj().T @ n_is1 @ final_state_bell))
n2 = (np.abs(final_state_bell.conj().T @ n_is2 @ final_state_bell))
c = (np.abs(final_state_bell.conj().T @ cc_op @ final_state_bell))
c_dag = (np.abs(final_state_bell.conj().T @ cc_dagger_op @ final_state_bell))
gamma = (np.abs(final_state_bell.conj().T @ gamma_op @ final_state_bell))
E = (np.abs(final_state_bell.conj().T @ H_analytical @ final_state_bell))
print('Bell state:')
print(f'n_1 = {n}, n_2 = {n2}, gamma = {gamma}, cc = {c}, cc_dag = {c_dag}, E = {E}')


### Analytical

In [7]:
# \begin{aligned}
# H &= \Delta\left( Y_1 Y_2 I_3 I_4 \right) \\ YYII and IYYI commute, YYII and IIYY commute, IXXI and YYII do not commute
# &+ \frac{t_c}{2} \left( I_1 X_2 X_3 I_4 + I_1 Y_2 Y_3 I_4 \right) \\ IXXI and IYYI commute
# &+ \Delta\left( I_1 I_2 Y_3 Y_4 \right)  \; . IIYY and YYII commute, IIYY and IYYI commute, IXXI and IIYY do not commute
# \end{aligned}
a = np.kron(np.kron(I,X),np.kron(X,I))
b = np.kron(np.kron(I,I),np.kron(Y,Y))

# Calculate the commutator
commutator = a @ b - b @ a

is_commuting = np.allclose(commutator, np.zeros_like(commutator))

if is_commuting:
    print("Matrices a and b commute.")
    print("Commutator a@b - b@a is (close to) the zero matrix:")
else:
    print("Matrices a and b do NOT commute.")
    print("Commutator a@b - b@a is:")

Matrices a and b do NOT commute.
Commutator a@b - b@a is:


### Qiskit

In [4]:
from qiskit.opflow import Zero, One, I, X, Y, Z

def compute_H(eps, t, delta):
    # Build the Hamiltonian using tensor products of Pauli matrices
    H = 0
    H += eps * (I ^ I)  # I ⊗ I
    H -= eps / 2 * (I ^ Z)  # I ⊗ Z
    H -= eps / 2 * (Z ^ I)  # Z ⊗ I
    H += (t + delta) / 2 * (X ^ X)  # X ⊗ X
    H += (t - delta) / 2 * (Y ^ Y)  # Y ⊗ Y
    
    return H
def n_op(num_sites):
    n_is = [
    1 / 2 * ((I ^ num_sites) - ((I ^ i) ^ Z ^ (I ^ (num_sites - i - 1))))
    for i in range(num_sites)
    ]
    return n_is

def cc_nb_op(num_sites, site1, site2):
    if abs(site1 - site2) != 1:
        raise ValueError('Error: Sites have to be nearest neighbour sites. Use corr_op for other sites.')
    sigma_plus = 0.5*(X + 1.0j*Y)
    if site1 < site2:
        cc_op = [ 
            (I ^ site1) ^ -sigma_plus ^ sigma_plus ^ (I ^ (num_sites - site1 - 2)) 
        ]
    elif site1 > site2:
        cc_op = [
            (I ^ site2) ^ sigma_plus ^ sigma_plus ^ (I ^ (num_sites - site2 - 2)) 
        ]

    return cc_op

from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

def compute_H_secondquant(eps, t, delta):
    second_quant_H = 0
    second_quant_H += (
                    FermionicOp("+_{}".format(0)) @ FermionicOp("-_{}".format(0))
                    + FermionicOp("+_{}".format(1)) @ FermionicOp("-_{}".format(1))
                ) * eps

    second_quant_H += (
                    FermionicOp("+_{}".format(0)) @ FermionicOp("-_{}".format(1))
                    + FermionicOp("+_{}".format(1)) @ FermionicOp("-_{}".format(0))
                ) * t
    second_quant_H += (
                    FermionicOp("+_{}".format(0)) @ FermionicOp("+_{}".format(1))
                    + FermionicOp("-_{}".format(1))
                    @ FermionicOp("-_{}".format(0))
                ) * delta

    jw_mapper = JordanWignerMapper
    pauli_H = JordanWignerMapper.map(jw_mapper, second_quant_H)
    return pauli_H

def compute_U(H, t):
    U = (H * t).exp_i()  # Apply the time evolution operator
    return U

# Parameterized hopping strengths
eps_param = Parameter("ε")
t_param = Parameter("t")
delta_param = Parameter("Δ")
H_param = compute_H(eps_param, t_param, delta_param)

# Select specific tau and tau_d values
H = H_param.bind_parameters({eps_param: eps, t_param: t, delta_param: delta})

# Initial state |00>
initial_state = One ^ Zero#(1 / np.sqrt(2))*(Zero ^ Zero) + (1 / np.sqrt(2))*(One ^ One)

# Time evolution operator
U_qiskit = compute_U(H, time)

# Apply the unitary operator to the initial state
final_state = U_qiskit @ initial_state

# Example of calculating expectation values (for n_is operator)
# Let's assume n_op returns an operator, and you want to calculate the expectation value.
n_is = n_op(2)
cc_is = cc_nb_op(2,0,1)
exp_n = (final_state.adjoint() @ n_is[0] @ final_state).eval()
exp_cc = (final_state.adjoint() @ cc_is[0] @ final_state).eval()

In [ ]:
def cc_edge_op(num_sites, site1, site2):
    sigma_plus = 0.5 * (X + 1.0j * Y)
    if site1 < site2:
        cc_op = [
            sigma_plus ^ (Z ^ (num_sites - 2)) ^ -sigma_plus
        ]
    elif site1 > site2:
        cc_op = [
            sigma_plus ^ (Z ^ (num_sites - 2)) ^ sigma_plus
        ]
            
    return cc_op

cc_edge_op(3, 0, 2)

[PauliSumOp(SparsePauliOp(['XZX', 'XZY', 'YZX', 'YZY'],
               coeffs=[-0.25+0.j  ,  0.  -0.25j,  0.  -0.25j,  0.25+0.j  ]), coeff=1.0)]

### Symbolic

In [26]:
import sympy as sp
from sympy.physics.quantum import TensorProduct

# Define symbolic parameters
eps, t, delta, tau = sp.symbols('varepsilon t t tau')
initial_state = np.kron(ket_1,ket_0)# (1/np.sqrt(2))*np.kron(ket_0,ket_0) + (1/np.sqrt(2))*np.kron(ket_1,ket_1)## #

# Define Pauli matrices and identity
I = sp.eye(2)
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])

# Define the symbolic Hamiltonian H1
H1 = (eps/2)*(TensorProduct(I, I) - TensorProduct(I, Z)) \
   + (eps/2)*(TensorProduct(I, I) - TensorProduct(Z, I)) \
   + ((t + delta)/2)*TensorProduct(X, X) \
   + ((t - delta)/2)*TensorProduct(Y, Y)

# Expand to 4x4 matrix
H1_matrix = H1.doit()

# Define time evolution operator U(tau) = exp(-i H tau)
U_matrix = (-sp.I * H1_matrix * tau).exp()

n_matrix_1 = (1/2)*(TensorProduct(I, I) - TensorProduct(Z, I))



# Convert to LaTeX
latex_H = sp.latex(H1_matrix)
latex_N = sp.latex(H1_matrix @ n_matrix_1)


# Write to a .tex file
with open("symbolic_H_and_U.tex", "w") as f:
    f.write(latex_H + "\n")
    f.write(latex_N)

print("LaTeX file 'symbolic_H_and_U.tex' written successfully.")

initial_state = np.kron(ket_1,ket_0)# (1/np.sqrt(2))*np.kron(ket_0,ket_0) + (1/np.sqrt(2))*np.kron(ket_1,ket_1)## #

display(Math(rf'U({time}) \; |10\rangle:'))
final_state_10 = U_matrix @ np.kron(ket_1,ket_0)

display(Math(rf'U({time}) \; \sqrt{1/2}|00\rangle + \sqrt{1/2}|11\rangle:'))
final_state_bell = U_matrix @ ((1/np.sqrt(2))*np.kron(ket_0,ket_0) + (1/np.sqrt(2))*np.kron(ket_1,ket_1))

final_state_bell.conj().T @ cc_is[0] @final_state_bell


LaTeX file 'symbolic_H_and_U.tex' written successfully.


<IPython.core.display.Math object>

<IPython.core.display.Math object>

AttributeError: 'MutableDenseMatrix' object has no attribute 'conj'